<a href="https://colab.research.google.com/github/LiadTssf/cloud-project/blob/main/targil8/Copy_of_cloud_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install firebase nltk
!pip install reportlab

In [ ]:
import re
import random
import nltk
from nltk.chat.util import Chat, reflections
from IPython.display import display, clear_output, FileLink
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ipywidgets as widgets
from firebase import firebase
from io import BytesIO
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Image
from reportlab.lib.units import inch
from google.colab import files


# Functions

In [ ]:
class OnShapeAnalyticsBot(Chat):
    def __init__(self, pairs, reflections, data):
        super().__init__(pairs, reflections)
        self.data = data

    def respond(self, message):
        for pattern, responses in self._pairs:
            match = re.match(pattern, message.rstrip(".!"))
            if match:
                response = random.choice(responses)
                if callable(response):
                    return response()
                else:
                    return self._wildcards(response, match)

        # If no match found, try to answer based on database info
        if "how many" in message.lower() and "projects" in message.lower():
            return f"There are {len(data_documents)} projects in the database."
        elif "how many" in message.lower() and "users" in message.lower():
            return f"There are {len(set(item.get('User') for item in self.data))} users in the database."
        elif "how many" in message.lower() and "actions" in message.lower():
            return f"There are {len(self.data)} total actions in the database."

        return "I'm not sure how to respond to that. Can you please rephrase your question?"

# Make sure to define the get_database_info function
def get_database_info():
    project_count = len(data_documents)
    user_count = len(set(item.get("User") for item in data))
    action_count = len(data)

    return f"There are {project_count} projects, {user_count} users, and {action_count} total actions in the database."

# Update the patterns
patterns = [
    (r'hi|hello|hey', ['Hello!', 'Hi there!', 'Hey!']),
    (r'how are you?', ['Im good, thank you!', 'Im doing well, thanks for asking.']),
    (r'what is your name?', ['You can call me OnShapeAnalyticsBot.', 'I go by the name OnShapeAnalyticsBot.']),
    (r'my name is (.*)', ['Nice to meet you, %1.', 'Hello, %1!']),
    (r'thank you (.*)', ['Youre welcome!', 'Happy to help!']),
    (r'What is the purpose of the system ?', ['The purpose of the system is to help software managers improve the teams performance by analyzing their work.']),
    (r'what\'s included in the software', ['The software includes an action counter, graphs for every action per worker/student, and a notification page for the manager to see important user activity or system notifications.']),
    (r'what is onshape|onshape|on shape', ['Onshape is a cloud-based CAD platform for product design and development. Learn more here: https://www.onshape.com/']),
    (r'how to filter the graph', ['You can filter the graph by clicking on the desirable data.']),
    (r'how to add a new team member', ['At this moment of development, we dont have this feature yet.']),
    (r'how to show important actions', ['Important actions are individual. You can filter them to your liking.']),
    (r'how to show who did the most actions', ['If you go to the team graph section, you can filter it with actions and see who contributed the most.']),
    (r'tell me about the database', [get_database_info]),
    (r'quit|exit|bye', ['Goodbye!', 'See you later!', 'Bye!']),
]

def run_chatbot():
    chatbot  = OnShapeAnalyticsBot(patterns, reflections, data)
    output = widgets.Output()
    input_box = widgets.Text(placeholder='Type your message here')
    send_button = widgets.Button(description="Send")

    chat_layout = widgets.VBox([output, widgets.HBox([input_box, send_button])])

    def on_send_button_clicked(b):
        user_input = input_box.value
        input_box.value = ''

        with output:
            print(f"You: {user_input}")
            response = chatbot.respond(user_input)
            print(f"OnShapeAnalyticsBot: {response}")

        if user_input.lower() in ['quit', 'exit', 'bye']:
            send_button.disabled = True
            input_box.disabled = True

    send_button.on_click(on_send_button_clicked)

    with output:
        print("Hello! I'm OnShapeAnalyticsBot. How can I help you today?")

    display(chat_layout)

# Update the chatbot button click handler
def on_chatbot_button_clicked(b):
    clear_output()
    run_chatbot()


## Database

In [ ]:
# Get data from firebase
def get_data_from_firebase(url, database, data):
    FBconn = firebase.FirebaseApplication(url, None)
    data = FBconn.get(database, data)
    return data

In [ ]:
# Function to get set array of onshape documents
def get_documents_set(data):
    documents = set()
    for item in data:
        document = item.get("Document")
        if document is not None:
            documents.add(document)
    return sorted(documents)

In [ ]:
# Function to get users associated with selected projects
def get_users_by_projects(data, selected_projects):
    users = set()
    for item in data:
        if item.get("Document") in selected_projects:
            users.add(item.get("User"))
    return users

## Buttons

In [ ]:
# Function to update team member tab content and navigate to it
def update_members(button):
    selected_projects = [checkbox.description for checkbox in project_checkboxes if checkbox.value]
    users = get_users_by_projects(data, selected_projects)
    user_checkboxes.clear()
    user_checkboxes.extend([
        widgets.Checkbox(
            value=False,
            description=str(user),
        ) for user in users])

    members_tab.children = [members_header] + user_checkboxes + [widgets.HBox([select_members_button], layout=widgets.Layout(justify_content='center', margin='20px 0 0 0'))]
    tabs.selected_index = 1  # Navigate to the "Select Team Member" tab

In [ ]:
def update_analytics(button):
    analytics_content.children = (
        [graph_type_dropdown,
         plots_output,
         widgets.HBox([generate_button, export_button], layout=widgets.Layout(justify_content='center', margin='20px 0 0 0'))]
    )

    with plots_output:
        clear_output(wait=True)
        fig, ax = plt.subplots()
        ax.pie([35, 25, 20, 20], labels=['A', 'B', 'C', 'D'], autopct='%1.1f%%')
        ax.set_title("No Graph Selected")
        plt.show()

    tabs.selected_index = 2  # Navigate to the "Analytics" tab

In [ ]:
def generate_graph(button):
    global generated_figures
    generated_figures.clear()  # Clear previous figures
    # clear outputs
    with plots_output:
        clear_output(wait=True)

    # Get current selections from UI elements
    selected_users = [checkbox.description for checkbox in user_checkboxes if checkbox.value]
    selected_projects = [checkbox.description for checkbox in project_checkboxes if checkbox.value]
    graph_type = graph_type_dropdown.value

    # Validate selections and perform graph generation
    if selected_users and selected_projects:

        #**************** Contribution Breakdown ******************
        if graph_type == 'Contribution Breakdown':
            contribution_breakdown(selected_users, selected_projects)

        #**************** Activity Timeline ******************
        elif graph_type == 'Activity Timeline':
            activity_timeline(selected_users, selected_projects)

        #**************** Task Distribution ******************
        elif graph_type == 'Task Distribution':
            task_distribution(selected_users, selected_projects)

        #**************** Performance Metrics ******************
        elif graph_type == 'Performance Metrics':
            performance_metrics(selected_users, selected_projects)

        #*********************** All Graphs *********************
        elif graph_type == 'All':
            all_graphs(selected_users, selected_projects)

        else:
            print("Invalid Graph Type selected.")

    else:
        print("Please select at least one user and one project.")





generated_figures = []

## Graphs

In [ ]:
def contribution_breakdown(selected_users, selected_projects):
    global generated_figures
    activity = {}
    for item in data:
        user = item.get("User")
        project = item.get("Document")
        if user in selected_users and project in selected_projects:
            month = int(item.get("Time").split('-')[1])
            activity.setdefault(user, [0]*12)[month - 1] += 1

    labels = list(activity.keys())
    sizes = np.sum(list(activity.values()), axis=1)

    fig, ax = plt.subplots()
    ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140)
    ax.set_title("Contribution Breakdown")
    generated_figures.append(fig)

    with plots_output:
        plt.show()

In [ ]:
def activity_timeline(selected_users, selected_projects):
    global generated_figures
    activity = {user: [0]*12 for user in selected_users}
    for item in data:
        user = item.get("User")
        project = item.get("Document")
        if user in selected_users and project in selected_projects:
            month = int(item.get("Time").split('-')[1])
            activity[user][month - 1] += 1

    fig, ax = plt.subplots()
    for user, activity_data in activity.items():
        ax.plot(activity_data, label=user)
    ax.set_xticks(range(12))
    ax.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
    ax.set_title("Activity Timeline")
    ax.legend()
    generated_figures.append(fig)

    with plots_output:
        plt.show()

In [ ]:
def task_distribution(selected_users, selected_projects):
    global generated_figures
    tasks = {}
    for item in data:
        user = item.get("User")
        project = item.get("Document")
        if user in selected_users and project in selected_projects:
            task_type = item.get("Description").split(' ')[0]  # Example extraction
            tasks.setdefault(user, {}).setdefault(task_type, 0)
            tasks[user][task_type] += 1

    # Convert tasks dictionary to a DataFrame
    df = pd.DataFrame(tasks).fillna(0).astype(int)

    # Calculate the sum of all actions and add it as a new row
    total_actions = df.sum(axis=0)
    df.loc['Total Actions'] = total_actions

    # Rename the index to "Actions"
    df.index.name = 'Actions'

    # Create a figure from the DataFrame
    fig, ax = plt.subplots(figsize=(12, 6))
    df.plot(kind='bar', ax=ax)
    ax.set_title("Task Distribution")
    ax.legend(title="Users")
    plt.tight_layout()
    generated_figures.append(fig)

    with plots_output:
        display(df)
        plt.show()

In [ ]:
def performance_metrics(selected_users, selected_projects):
    global generated_figures
    quality_metrics = {user: {'Actions Completed': 0, 'Changes Made': 0, 'Fixes Applied': 0} for user in selected_users}

    for item in data:
        user = item.get("User")
        project = item.get("Document")
        if user in selected_users and project in selected_projects:
            action = item.get("Description").split(' ')[0]  # Extract action keyword

            if action in ['Change', 'Edit', 'Fix', 'Update', 'Animate', 'Suppress', 'Create']:
                quality_metrics[user]['Actions Completed'] += 1

            if action in ['Change', 'Edit', 'Fix', 'Update', 'Animate', 'Move']:
                quality_metrics[user]['Changes Made'] += 1

            if action in ['Fix', 'Unfix']:
                quality_metrics[user]['Fixes Applied'] += 1

    metrics = ['Actions Completed', 'Changes Made', 'Fixes Applied']
    # Plotting quality metrics
    fig, ax = plt.subplots()
    for metric in metrics:
        ax.bar(quality_metrics.keys(), [quality_metrics[user][metric] for user in quality_metrics.keys()], label=metric)
    ax.set_title("Performance Metrics")
    ax.legend()
    generated_figures.append(fig)

    with plots_output:
        plt.show()

In [ ]:
def all_graphs(selected_users, selected_projects):
    contribution_breakdown(selected_users, selected_projects)
    activity_timeline(selected_users, selected_projects)
    task_distribution(selected_users, selected_projects)
    performance_metrics(selected_users, selected_projects)

In [ ]:
from reportlab.platypus import SimpleDocTemplate, Image, Table, TableStyle, Paragraph
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors

def download_graphs(button):
    global generated_figures

    if not generated_figures:
        print("No graphs were generated. Please generate graphs before attempting to download.")
        return

    # Create a BytesIO object to store the PDF
    buffer = BytesIO()

    # Create the PDF document
    doc = SimpleDocTemplate(buffer, pagesize=letter)
    elements = []

    # Iterate through all the generated figures
    for fig in generated_figures:
        # Save the figure to a BytesIO object
        img_buffer = BytesIO()
        fig.savefig(img_buffer, format='png', dpi=300, bbox_inches='tight')
        img_buffer.seek(0)

        # Create an Image object and add it to the elements list
        img = Image(img_buffer)
        img.drawHeight = 6*inch
        img.drawWidth = 8*inch
        elements.append(img)

    # Build the PDF
    doc.build(elements)

    # Move the buffer's cursor to the beginning
    buffer.seek(0)

    # Generate a filename
    filename = f"onshape_analytics_{len(generated_figures)}_graphs.pdf"

    # Write the PDF content to a file
    with open(filename, 'wb') as f:
        f.write(buffer.getvalue())

    # Use google.colab.files to initiate the download
    files.download(filename)

    print(f"PDF file '{filename}' has been created and downloaded.")

    # Clear the generated figures list
    generated_figures.clear()

# Program

## Initialisation

In [ ]:
data = get_data_from_firebase('https://test100000-ccdb4-default-rtdb.asia-southeast1.firebasedatabase.app/', '/team_data2', 'data')
data_documents = get_documents_set(data)

## Widgets

In [ ]:
# stylesheet
style = widgets.HTML("""
    <style>
      HTML {
        height: 100vh;
      }
      body {
        font-family: 'Times New Roman', serif;
        background-color: #ffffff;

      }
      ul {
        padding-left: 20px;
        margin-top: 10px;
        margin-bottom: 10px;
        list-style-type: disc;
        justify_content: space-between;
      }
      .jupyter-widgets.widget-tab > .p-TabBar .p-TabBar-tab {
        flex: 1 1 var(--jp-widgets-horizontal-tab-width);
        background-color: #007bff; /* Blue tab background */
        color: white; /* White tab text */
        font-weight: bold;
      }
      .jupyter-widgets.widget-tab > .p-TabBar .p-TabBar-tab.p-mod-current {
        background-color: #0056b3; /* Darker blue for active tab */
      }
      li {
        font-size: 20px !important;
        font-weight: bold;
        margin-bottom: 5px;
        padding: 10px;
        gap: 10px;
        background-color: #ffffff; /* White background for list items */
        border: solid 1px #dcdcdc; /* Light gray border for list items */
        border-radius: 5px; /* Rounded corners for list items */
      }
      h3 {
        text-decoration: underline;
      }
      span {
        font-size: 20px !important;
      }
      select, option {
        font-family: 'Times New Roman', serif;
        font-size: 20px !important;
      }
      .widget-label {
        font-size: 20px;
      }
      .widget-button {
        background-color: #007bff;
        color: white;
        height: 50px;
        font-size: 24px;
        font-weight: bold;
        border: none;
        border-radius: 5px;
        padding: 10px;
        cursor: pointer;
      }
      .widget-button:hover {
        background-color: #0056b3;
      }
      .lm-Widget.p-Widget.lm-Panel.p-Panel.p-TabPanel-tabContents.widget-tab-contents {
        background: rgb(8,129,113);
        background: linear-gradient(90deg, rgba(8,129,113,1) 0%, rgba(237,237,237,1) 67%);
      }
    </style>
""")

In [ ]:
## projects tab

# header
project_header = widgets.HTML("<h2>Select Projects</h2>")

# checkboxs
project_checkboxes = [
    widgets.Checkbox(
        value=False,
        description=str(doc),
    ) for doc in data_documents if doc is not None
]

# select button
select_projects_button = widgets.Button(
    description="Next",
)

select_projects_button.on_click(update_members)

# layout
projects_tab = widgets.VBox([
        project_header,
        widgets.VBox(project_checkboxes),
        widgets.HBox([select_projects_button], layout=widgets.Layout(justify_content='center', margin='20px 0 0 0'))
    ])

In [ ]:
## members tab

user_checkboxes = []

# start empty layout
members_tab = widgets.VBox([widgets.Label("Select projects to continue")])

# header
members_header = widgets.HTML("<h2>Select Team Members</h2>")

# select button
select_members_button = widgets.Button(
    description="Next",
)
select_members_button.on_click(update_analytics)

# full layout in function update_members()

In [ ]:
## analytics tab

plots_output = widgets.Output()

# header
members_header = widgets.HTML("<h2>Analytics</h2>")

# dropdown menu
graph_type_dropdown = widgets.Dropdown(
    options=['Contribution Breakdown', 'Activity Timeline', 'Task Distribution', 'Performance Metrics', 'All'],
    description='Select Graph Type:',
    style={'description_width': 'initial'}
)

# generate button
generate_button = widgets.Button(
    description="Generate",
)
# generate button
export_button = widgets.Button(
    description="export",
)
export_button.on_click(download_graphs)
generate_button.on_click(generate_graph)

# start empty layout
analytics_content = widgets.VBox([widgets.Label("Select projects and members to continue")])

# full layout in function update_analytics()

## Main

In [ ]:
# header
header = widgets.HTML("<h1>OnShape Analytics Program</h1>")

# tabs
tabs = widgets.Tab()
tabs.children = [
    projects_tab,
    members_tab,
    analytics_content
]
tabs.set_title(0, 'Select Projects')
tabs.set_title(1, 'Select Members')
tabs.set_title(2, 'Analytics')

chatbot_button = widgets.Button(
    description="Start Chatbot",
    button_style='info'
)
# layout and display
app_layout = widgets.VBox([style, header, tabs, chatbot_button])


chatbot_button.on_click(on_chatbot_button_clicked)



display(app_layout)